<a href="https://colab.research.google.com/github/guiraposo/ML4Gravity/blob/main/ML4Waveforms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Waveform generation and detetection

Notebook based on the [MachineLearningForStrongGravity](https://github.com/raimonluna/) course given by Raimon Luna.

<br>

### In this notebook, we will see:

1. How to reduce the dimensionality of data by finding an appropriate basis of vectors.
2. How autoencoders "compress" the data by automatically identifying the governing parameters.
3. How we can use GANs to generate samples from the same distribution as the training data.<br>Original paper:<br>

 - Felipe F. Freitas, Carlos A. R. Herdeiro, António P. Morais, António Onofre, Roman Pasechnik, Eugen Radu, Nicolas Sanchis-Gual, Rui Santos, <i>Generating gravitational waveform libraries of exotic compact binaries with deep learning</i>, 2022. https://arxiv.org/abs/2203.01267

<br>

In [6]:
# Import relevant libraries and modules
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.decomposition import PCA, TruncatedSVD
import numpy as np

import matplotlib.pyplot as plt
from IPython import display

# Synthetic Waveforms:

Let us start by generating a dataset with some toy waveforms. We choose a generator of the following type:

$$\psi = \sin(2 \pi k x)\, e^{-8x^2}, $$

representing a sinusoidal function with wave number $k$, modulated by a gaussian profile. For simplicity we shall consider only one parameter ($k$), but in other sections we shall try to extend this to additional parameters.

In [5]:
x       = np.linspace(-1, 1, 64, dtype = np.float32)
randoms = 5*np.random.rand(100)
data    = torch.tensor(np.array([    np.sin(2 * k * np.pi * x) * np.exp(- 8 * x**2 ) for k in randoms]))

data_train, data_test = data[:60, :], data[60:, :]